Imports

In [1]:
import numpy 
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2 
import os
import random
import uuid

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, MaxPooling2D,Input,Flatten,Dense

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
     tf.config.experimental.set_memory_growth(gpu,True)

Paths

In [4]:
pos = os.path.join('positive')
neg = os.path.join('negative')
anc = os.path.join('anchor')

In [5]:
os

<module 'os' from 'C:\\Users\\Com\\anaconda3\\envs\\tFlow\\lib\\os.py'>

In [6]:
!tar -xf lfw.tgz

tar: Error opening archive: Failed to open 'lfw.tgz'


In [7]:
# for dir in os.listdir('lfw'):
#     for file in os.listdir(os.path.join('lfw',dir)):
#         old_path =  os.path.join('lfw',dir,file)
#         new_path = os.path.join(neg, file)
#         os.replace(old_path, new_path)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'lfw'

In [ ]:
cap = cv2.VideoCapture(0)
a_count = 0 
p_count = 0
while cap.isOpened(): 
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    if cv2.waitKey(1) & 0XFF == ord('a'):
        print('frame no {}'.format(a_count))
        imgname = os.path.join(anc, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)
        a_count+=1
    if cv2.waitKey(1) & 0XFF == ord('p'):
        print('frame no {}'.format(p_count))
        imgname = os.path.join(pos, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)
        p_count+=1
    cv2.imshow('Image Collection', frame)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

tf.data.Dataset.list_files creates pipeline for all files in directory 

In [9]:
anchor  = tf.data.Dataset.list_files('anchor'+'\*.jpg').take(500)
positive  = tf.data.Dataset.list_files('positive'+'\*.jpg').take(500)
negative  = tf.data.Dataset.list_files('negative'+'\*.jpg').take(500)

In [10]:
print(type(anchor))

<class 'tensorflow.python.data.ops.dataset_ops.TakeDataset'>


In [11]:
dir_test = anchor.as_numpy_iterator()

In [12]:
dir_test.next()

b'anchor\\fb0f147f-079d-11ef-8171-7066553b5cdc.jpg'

In [13]:
def normalization(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img,(100,100))
    img = img/255.0
    return img

In [16]:
nor = normalization('anchor\\e031edb8-079d-11ef-9607-7066553b5cdc.jpg')

In [17]:
nor

<tf.Tensor: shape=(100, 100, 3), dtype=float32, numpy=
array([[[0.6735294 , 0.66568625, 0.6696078 ],
        [0.6843137 , 0.6666667 , 0.6696078 ],
        [0.6997549 , 0.67034316, 0.6644608 ],
        ...,
        [0.8264706 , 0.8254902 , 0.8362745 ],
        [0.81960785, 0.8156863 , 0.8382353 ],
        [0.83137256, 0.827451  , 0.8509804 ]],

       [[0.67107844, 0.66519606, 0.6661765 ],
        [0.67009807, 0.6632353 , 0.65686274],
        [0.6884804 , 0.65882355, 0.6627451 ],
        ...,
        [0.8281863 , 0.8262255 , 0.84583336],
        [0.80784315, 0.80588233, 0.8276961 ],
        [0.8129902 , 0.8159314 , 0.8384804 ]],

       [[0.67745095, 0.68039215, 0.6637255 ],
        [0.6813725 , 0.6745098 , 0.6696078 ],
        [0.6742647 , 0.6526961 , 0.6639706 ],
        ...,
        [0.8208333 , 0.8247549 , 0.84436274],
        [0.8252451 , 0.8269608 , 0.85318625],
        [0.81764704, 0.8264706 , 0.85490197]],

       ...,

       [[0.13137256, 0.11470588, 0.13137256],
        [0.06

from_tensor_slices gives slices of numpy array as object 

In [18]:
Positive_imgs = tf.data.Dataset.zip((anchor,positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
Negative_imgs = tf.data.Dataset.zip((anchor,negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))

In [19]:
Data = Positive_imgs.concatenate(Negative_imgs)

In [20]:
sample = Data.as_numpy_iterator()

In [21]:
example = sample.next()

In [22]:
example

(b'anchor\\fae8cdf8-079d-11ef-9e2c-7066553b5cdc.jpg',
 b'positive\\5af4530c-079e-11ef-9673-7066553b5cdc.jpg',
 1.0)

In [23]:
def preprocess_join(input_img, val_img, label):
    return (normalization(input_img),normalization(val_img),label)

In [24]:
Data = Data.map(preprocess_join)
Data = Data.cache()
Data.shuffle(buffer_size= 1024)

<ShuffleDataset shapes: ((100, 100, None), (100, 100, None), ()), types: (tf.float32, tf.float32, tf.float32)>

In [25]:
data =  Data.as_numpy_iterator()
d = data.next()

In [26]:
d

(array([[[0.6431373 , 0.627451  , 0.62352943],
         [0.6431373 , 0.627451  , 0.6254902 ],
         [0.64240193, 0.62671566, 0.6306372 ],
         ...,
         [0.7703431 , 0.7727941 , 0.79019606],
         [0.76862746, 0.779902  , 0.80539215],
         [0.7678922 , 0.7796569 , 0.814951  ]],
 
        [[0.6460784 , 0.6284314 , 0.6254902 ],
         [0.6497549 , 0.63210785, 0.6311275 ],
         [0.64754903, 0.62990195, 0.63480395],
         ...,
         [0.75490195, 0.76862746, 0.79068625],
         [0.7556372 , 0.7752451 , 0.7987745 ],
         [0.74485296, 0.76642156, 0.78995097]],
 
        [[0.6519608 , 0.6284314 , 0.6284314 ],
         [0.6573529 , 0.6338235 , 0.6343137 ],
         [0.6497549 , 0.6306372 , 0.63039213],
         ...,
         [0.75392157, 0.7735294 , 0.7970588 ],
         [0.7639706 , 0.7894608 , 0.8129902 ],
         [0.7441176 , 0.7745098 , 0.7892157 ]],
 
        ...,
 
        [[0.6732843 , 0.65955883, 0.6394608 ],
         [0.7012255 , 0.68333334, 0.67426

In [27]:
d[2]

1.0

In [28]:
train_data = Data.take(round(len(Data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [29]:
td_sample = train_data.as_numpy_iterator()

In [30]:
td_samp = td_sample.next()

In [31]:
len(td_samp[0])

16

In [32]:
test_data = Data.skip(round(len(Data)*.7))
test_data = Data.take(round(len(Data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [33]:
#embedding layers

In [34]:
def make_embed():
    input_layer = Input(shape= (100,100,3))
    cl1 = Conv2D(64,(10,10),activation= 'relu')(input_layer)
    mp1 = MaxPooling2D(64,(2,2), padding = 'same')(cl1)
    
    cl2 = Conv2D(128,(7,7),activation= 'relu')(mp1)
    mp2 = MaxPooling2D(64,(2,2), padding = 'same')(cl2)
    
    cl3 = Conv2D(128,(4,4),activation= 'relu')(mp2)
    mp3 = MaxPooling2D(64,(2,2), padding = 'same')(cl3)
    
    cl4 = Conv2D(256,(4,4),activation= 'relu')(mp3)
    f1 = Flatten()(cl4)
    d1 = Dense(4096,activation = 'sigmoid')(f1)
    
    return Model(inputs = input_layer, outputs =d1 ,name ='embedding' )
    

In [35]:
embed = make_embed()

In [36]:
embed.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 46, 46, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 20, 20, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 17, 128)       26

In [38]:
class L1Dist(Layer):
    def __init__(self,**kwargs):
        super().__init__()
    def call(self,input_embed,val_embed):
        return tf.math.abs(input_embed-val_embed)

In [39]:
l1 = L1Dist()

In [40]:
input_image = Input(name='input_img', shape=(100,100,3))
validation_image = Input(name='validation_img', shape=(100,100,3))

In [41]:
inp_embedding = embed(input_image)
val_embedding = embed(validation_image)

In [42]:
siamese_layer = L1Dist()

In [43]:
def make_siam_model():
    inp_img = Input(name= 'inp_img' , shape = (100,100,3))
    val_img = Input(name= 'val_img' , shape = (100,100,3))
    Siam_layer  = L1Dist()
    Siam_layer._name = 'distance'
    distance = Siam_layer(embed(inp_img),embed(val_img))
    classifier = Dense(1, activation = 'sigmoid')(distance)
    return Model(inputs = [inp_img,val_img], outputs =classifier ,name= 'Sesmie' )

In [44]:
siam_model = make_siam_model()

In [45]:
siam_model.summary()

Model: "Sesmie"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inp_img (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 val_img (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['inp_img[0][0]',                
                                                                  'val_img[0][0]']           

In [46]:
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [47]:
opt = tf.keras.optimizers.Adam(1e-4) # 0.0001

In [48]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siam_model=siam_model)

In [49]:
test_batch = train_data.as_numpy_iterator()

In [50]:
batch_1 = test_batch.next()

In [51]:
X = batch_1[:2]

In [52]:
Y = batch_1[2]

In [53]:
@tf.function
def train_step(batch):
    
    # Record all of our operations 
    with tf.GradientTape() as tape:     
        # Get anchor and positive/negative image
        X = batch[:2]
        # Get label
        y = batch[2]
        
        # Forward pass
        yhat = siam_model(X, training=True)
        # Calculate loss
        loss = binary_cross_loss(y, yhat)
    print(loss)
        
    # Calculate gradients
    grad = tape.gradient(loss, siam_model.trainable_variables)
    
    # Calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siam_model.trainable_variables))
    
    # Return loss
    return loss

In [54]:
def train(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run train step here
            train_step(batch)
            progbar.update(idx+1)
        
        # Save checkpoints
        if epoch % 10 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

In [55]:
EPOCHS = 50

In [56]:
train(train_data, EPOCHS)


 Epoch 1/50
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
44/44 [==============================] - 31s 442ms/step

 Epoch 2/50
44/44 [==============================] - 17s 401ms/step

 Epoch 3/50
44/44 [==============================] - 18s 403ms/step

 Epoch 4/50
44/44 [==============================] - 18s 403ms/step

 Epoch 5/50
44/44 [==============================] - 18s 405ms/step

 Epoch 6/50
44/44 [==============================] - 18s 406ms/step

 Epoch 7/50
44/44 [==============================] - 18s 407ms/step

 Epoch 8/50
44/44 [==============================] - 18s 407ms/step

 Epoch 9/50
44/44 [==============================] - 18s 408ms/step

 Epoch 10/50
44/44 [==============================] - 18s 406ms/step

 Epoch 11/50
44/44 [==============================] - 18s 409ms/step

 Epoch 12/50
44/44 [==============================] - 18s 409ms/step

 Epoch 13/50
4

In [129]:
# type(train_data)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [58]:
tf.data.experimental.save(test_data , 'Sesmi/test_data')

In [133]:
print(tf.__version__)

2.7.0


In [57]:
len(tf.config.list_physical_devices('GPU'))>0


True